## 針對 Double Error Correction(DEC)做Synthesizer(包含1bit 2bits AWE)
### python(.py) to verilog(.v)

#### Define function: (Not including 2's complement range)

In [1]:
import math
import numpy as np


def decimal_to_binary(decimal_number):
    if decimal_number == 0:
        return "0"
    # 處理正數：直接轉換
    if decimal_number > 0:
        return bin(decimal_number)[2:]  # 移除 "0b" 前綴
       # 處理負數：自動計算所需位元數（2's complement）
    abs_value = abs(decimal_number)
    bits = abs_value.bit_length() + 1  # 負數多一個符號位元
    two_complement_value = (1 << bits) + decimal_number
    
    return bin(two_complement_value)[2:]  # 移除 "0b" 前綴

def bits_needed(number):
    return number.bit_length()


def bits_range(bits, signed=False):     ## 可選擇設定 有號數 或 無號數
    if bits <= 0:
        return "位元數必須為正整數！"
    
    if signed:
        # 二補數範圍: -2^(bits-1) 到 2^(bits-1)-1
        min_value = -(2**(bits - 1))
        max_value = 2**(bits - 1) - 1
    else:
        # 無符號範圍: 0 到 2^bits - 1
        min_value = 0
        max_value = 2**bits - 1

    return min_value, max_value

## 把重複的 AWE 去除
def remove_duplicates(input_list):
    unique_list = []
    for number in input_list:
        if number not in unique_list:
            unique_list.append(number)
    return (unique_list)

def get_sign(number):
    return 1 if number > 0 else -1 if number < 0 else 0

print("一個2補數所需要的位元數",bits_needed(120))
print("一bits數可表示的2補數的範圍",bits_range(bits_needed(120)))
print("一個2補數表達為二進制為:",decimal_to_binary(-8))

一個2補數所需要的位元數 7
一bits數可表示的2補數的範圍 (0, 127)
一個2補數表達為二進制為: 11000


### 查找表範例

In [2]:
def create_mapping(source_list, address_map):
    """
    建立值到位址和位址到值的雙向映射。
    若 address_map 中某位址沒有對應值，則該位址的值為 0。
    
    參數:
    source_list (list): 原始資料列表。
    address_map (list): 位址映射列表。
    
    回傳:
    tuple: (值到位址的映射, 位址到值的映射)
    """
    # 初始化值到位址的映射
    value_to_address = {value: addr for value, addr in zip(source_list, address_map)}
    
    # 初始化位址到值的映射，未指定的位址預設為 0
    max_address = max(address_map)  # 確保涵蓋所有位址
    address_to_value = {addr: 0 for addr in range(max_address + 1)}  # 預設為 0
    for value, addr in zip(source_list, address_map):
        address_to_value[addr] = value  # 更新有指定值的位址

    return value_to_address, address_to_value

# 測試數據
source_list = [10, 20, 30]
address_map = [2, 0, 4]

# 建立映射
value_to_address, address_to_value = create_mapping(source_list, address_map)

# 顯示結果
print("值到位址的映射:", value_to_address)
print("位址到值的映射:", address_to_value)


# 測試數據
source_list = [10, 20, 30, 40]
address_map = [2, 0, 3, 1]

# 建立映射
value_to_address, address_to_value = create_mapping(source_list, address_map)

# 測試查詢
print("值到位址:", value_to_address)  # 查詢 10 -> 2, 20 -> 0
print("位址到值:", address_to_value)  # 查詢 2 -> 10, 0 -> 20

# 範例查詢
value = 10
address = 2
print(f"輸入值 {value}, 對應位址: {value_to_address[value]}")
print(f"輸入位址 {address}, 對應值: {address_to_value[address]}")

值到位址的映射: {10: 2, 20: 0, 30: 4}
位址到值的映射: {0: 20, 1: 0, 2: 10, 3: 0, 4: 30}
值到位址: {10: 2, 20: 0, 30: 3, 40: 1}
位址到值: {0: 20, 1: 40, 2: 10, 3: 30}
輸入值 10, 對應位址: 2
輸入位址 2, 對應值: 10


### 選取特定的位元數:

In [3]:
### 輸入數值為data, ANcode的A為AN
### 乘完A後的數值為 input
data = (1 << 52) - 1     ## 
A = 50861
input = data*A
bits = bits_needed(input)
print("AN資料的位元數:", bits)
print(bits_needed(A))
print(bits_needed(data))

AN資料的位元數: 68
16
52


### 產生1-bit AWE (Arithmetic-Weight Error)

In [4]:
AWE_1=[]
e1=0     ## 錯誤數量
e1_data=[]
h = []   #位置
for i in range(0, bits):
    e1_data.insert(e1 , input + 2**i)
    AWE_1.insert(e1, 2**i)
    e1 = e1 + 1
    e1_data.insert(e1 , input - 2**i)
    AWE_1.insert(e1, -2**i)
    e1 = e1 + 1
    h.insert(i,i+1)


h_awe = [val for num in h for val in (num, -num)]
print(bits)
print(input)
print("產生1-bit AWE:",AWE_1)
print(h_awe)
print("算術錯誤的資料:", e1_data)
print(e1)
print(np.size(AWE_1))

68
229057580647690746195
產生1-bit AWE: [1, -1, 2, -2, 4, -4, 8, -8, 16, -16, 32, -32, 64, -64, 128, -128, 256, -256, 512, -512, 1024, -1024, 2048, -2048, 4096, -4096, 8192, -8192, 16384, -16384, 32768, -32768, 65536, -65536, 131072, -131072, 262144, -262144, 524288, -524288, 1048576, -1048576, 2097152, -2097152, 4194304, -4194304, 8388608, -8388608, 16777216, -16777216, 33554432, -33554432, 67108864, -67108864, 134217728, -134217728, 268435456, -268435456, 536870912, -536870912, 1073741824, -1073741824, 2147483648, -2147483648, 4294967296, -4294967296, 8589934592, -8589934592, 17179869184, -17179869184, 34359738368, -34359738368, 68719476736, -68719476736, 137438953472, -137438953472, 274877906944, -274877906944, 549755813888, -549755813888, 1099511627776, -1099511627776, 2199023255552, -2199023255552, 4398046511104, -4398046511104, 8796093022208, -8796093022208, 17592186044416, -17592186044416, 35184372088832, -35184372088832, 70368744177664, -70368744177664, 140737488355328, -14073748

### 產生SEC的模數:

In [5]:
SEC_R = []
AWE1_R= []
for m in range(0,e1):
    mod = e1_data[m] % A
    SEC_R.append(mod)

for n in range(0,e1):
    mod_awe = AWE_1[n] % A
    AWE1_R.append(mod_awe)


print("SEC的模數:\n",SEC_R)
print("AWE1的模數:\n",SEC_R)
print("AWE1的模數是否等於SEC模數:", SEC_R == AWE1_R)
print("SEC的大小:",np.size(SEC_R))

SEC的模數:
 [1, 50860, 2, 50859, 4, 50857, 8, 50853, 16, 50845, 32, 50829, 64, 50797, 128, 50733, 256, 50605, 512, 50349, 1024, 49837, 2048, 48813, 4096, 46765, 8192, 42669, 16384, 34477, 32768, 18093, 14675, 36186, 29350, 21511, 7839, 43022, 15678, 35183, 31356, 19505, 11851, 39010, 23702, 27159, 47404, 3457, 43947, 6914, 37033, 13828, 23205, 27656, 46410, 4451, 41959, 8902, 33057, 17804, 15253, 35608, 30506, 20355, 10151, 40710, 20302, 30559, 40604, 10257, 30347, 20514, 9833, 41028, 19666, 31195, 39332, 11529, 27803, 23058, 4745, 46116, 9490, 41371, 18980, 31881, 37960, 12901, 25059, 25802, 50118, 743, 49375, 1486, 47889, 2972, 44917, 5944, 38973, 11888, 27085, 23776, 3309, 47552, 6618, 44243, 13236, 37625, 26472, 24389, 2083, 48778, 4166, 46695, 8332, 42529, 16664, 34197, 33328, 17533, 15795, 35066, 31590, 19271, 12319, 38542, 24638, 26223, 49276, 1585, 47691, 3170, 44521, 6340, 38181, 12680]
AWE1的模數:
 [1, 50860, 2, 50859, 4, 50857, 8, 50853, 16, 50845, 32, 50829, 64, 50797, 128, 50733

### 建立只能做SEC的LUT: Mapping (+-)location to (SEC_R, deltaQ1)

In [6]:
## mapping (+-)location to R
h_to_R, R_to_h = create_mapping(h_awe, SEC_R) 

### 產生2-bit AWE (Arithmetic-Weight Error)

In [7]:
AWE_2=[]
e2=0     ## 錯誤數量
p=0      ## 錯誤位置
e2_data=[]
for i in range(0, bits):
    for j in range(i+1, bits):
        e2_data.insert(e2 , (input + 2**i + 2**j))
        AWE_2.insert(e2 , (2**i + 2**j))        
        print(f"第 {e2} 筆錯誤資料: {e2_data[e2]}")
        print(f"第 {e2} 筆算術錯誤: {AWE_2[e2]}")
        e2 = e2 + 1
        e2_data.insert(e2 , (input + 2**i - 2**j))
        AWE_2.insert(e2 , (2**i - 2**j))        
        print(f"第 {e2} 筆錯誤資料: {e2_data[e2]}")
        print(f"第 {e2} 筆算術錯誤: {AWE_2[e2]}")
        e2 = e2 + 1
        e2_data.insert(e2 , (input - 2**i + 2**j))
        AWE_2.insert(e2 , (-2**i + 2**j))                
        print(f"第 {e2} 筆錯誤資料: {e2_data[e2]}")
        print(f"第 {e2} 筆算術錯誤: {AWE_2[e2]}")
        e2 = e2 + 1
        e2_data.insert(e2 , (input - 2**i -2**j))
        AWE_2.insert(e2 , (-2**i - 2**j))
        print(f"第 {e2} 筆錯誤資料: {e2_data[e2]}")
        print(f"第 {e2} 筆算術錯誤: {AWE_2[e2]}")
        e2 = e2 + 1
        p = p + 1


print(e2)
print(p)
print("產生2-bits AWE:",AWE_2)
print("產生2-bits 錯誤資料:",e2_data)

第 0 筆錯誤資料: 229057580647690746198
第 0 筆算術錯誤: 3
第 1 筆錯誤資料: 229057580647690746194
第 1 筆算術錯誤: -1
第 2 筆錯誤資料: 229057580647690746196
第 2 筆算術錯誤: 1
第 3 筆錯誤資料: 229057580647690746192
第 3 筆算術錯誤: -3
第 4 筆錯誤資料: 229057580647690746200
第 4 筆算術錯誤: 5
第 5 筆錯誤資料: 229057580647690746192
第 5 筆算術錯誤: -3
第 6 筆錯誤資料: 229057580647690746198
第 6 筆算術錯誤: 3
第 7 筆錯誤資料: 229057580647690746190
第 7 筆算術錯誤: -5
第 8 筆錯誤資料: 229057580647690746204
第 8 筆算術錯誤: 9
第 9 筆錯誤資料: 229057580647690746188
第 9 筆算術錯誤: -7
第 10 筆錯誤資料: 229057580647690746202
第 10 筆算術錯誤: 7
第 11 筆錯誤資料: 229057580647690746186
第 11 筆算術錯誤: -9
第 12 筆錯誤資料: 229057580647690746212
第 12 筆算術錯誤: 17
第 13 筆錯誤資料: 229057580647690746180
第 13 筆算術錯誤: -15
第 14 筆錯誤資料: 229057580647690746210
第 14 筆算術錯誤: 15
第 15 筆錯誤資料: 229057580647690746178
第 15 筆算術錯誤: -17
第 16 筆錯誤資料: 229057580647690746228
第 16 筆算術錯誤: 33
第 17 筆錯誤資料: 229057580647690746164
第 17 筆算術錯誤: -31
第 18 筆錯誤資料: 229057580647690746226
第 18 筆算術錯誤: 31
第 19 筆錯誤資料: 229057580647690746162
第 19 筆算術錯誤: -33
第 20 筆錯誤資料: 229057580647690746260
第 20 筆算術

### 合併所有AWE, 並去除重複的AWE

In [8]:
AWE = AWE_1 + AWE_2
e_total = e1_data + e2_data
print("還未去除重複error data的數量:",np.size(e_total))

## 把重複的 AWE 及 error_data 去除
new_AWE = remove_duplicates(AWE)
new_e_total = remove_duplicates(e_total)
error_total_count = np.size(new_e_total)
print("已去除重複error data的數量:",error_total_count)
print("已去除重複AWE的數量:",np.size(new_AWE))
print("所有的AWE為:\n",new_AWE)
print("所有算術錯誤的資料:\n", new_e_total)


還未去除重複error data的數量: 9248
已去除重複error data的數量: 8982
已去除重複AWE的數量: 8982
所有的AWE為:
 [1, -1, 2, -2, 4, -4, 8, -8, 16, -16, 32, -32, 64, -64, 128, -128, 256, -256, 512, -512, 1024, -1024, 2048, -2048, 4096, -4096, 8192, -8192, 16384, -16384, 32768, -32768, 65536, -65536, 131072, -131072, 262144, -262144, 524288, -524288, 1048576, -1048576, 2097152, -2097152, 4194304, -4194304, 8388608, -8388608, 16777216, -16777216, 33554432, -33554432, 67108864, -67108864, 134217728, -134217728, 268435456, -268435456, 536870912, -536870912, 1073741824, -1073741824, 2147483648, -2147483648, 4294967296, -4294967296, 8589934592, -8589934592, 17179869184, -17179869184, 34359738368, -34359738368, 68719476736, -68719476736, 137438953472, -137438953472, 274877906944, -274877906944, 549755813888, -549755813888, 1099511627776, -1099511627776, 2199023255552, -2199023255552, 4398046511104, -4398046511104, 8796093022208, -8796093022208, 17592186044416, -17592186044416, 35184372088832, -35184372088832, 70368744177664, -7

### 所有AWE數量: $4*C^{bits}_{2}$ $+$ $2*C^{bits}_{1}$  $-$ $2*(bits-1)$ - $2*(bits-2)$
* $2*(bits-1)$: 所有的 single AWE都可以分解成已重複的 double AWE, 除了MSB例外
* EX: $(+,0)$ 只能分解成 $(0,++)$ , $(-,0)$ 只能分解成 $(0,--)$
* $2*(bits-2)$: double AWE重複的個數
* EX: $(+,0,-)$ = $(0,+,+)$ and $(-,0,+)$ = $(0,-,-)$

### 產生DEC,SEC的模數:

In [9]:
remainder = []
for m in range(0,error_total_count):
    mod = new_e_total[m] % A
    remainder.append(mod)

print("錯誤資料對AN取完模數:\n",remainder)
print(np.size(remainder))

錯誤資料對AN取完模數:
 [1, 50860, 2, 50859, 4, 50857, 8, 50853, 16, 50845, 32, 50829, 64, 50797, 128, 50733, 256, 50605, 512, 50349, 1024, 49837, 2048, 48813, 4096, 46765, 8192, 42669, 16384, 34477, 32768, 18093, 14675, 36186, 29350, 21511, 7839, 43022, 15678, 35183, 31356, 19505, 11851, 39010, 23702, 27159, 47404, 3457, 43947, 6914, 37033, 13828, 23205, 27656, 46410, 4451, 41959, 8902, 33057, 17804, 15253, 35608, 30506, 20355, 10151, 40710, 20302, 30559, 40604, 10257, 30347, 20514, 9833, 41028, 19666, 31195, 39332, 11529, 27803, 23058, 4745, 46116, 9490, 41371, 18980, 31881, 37960, 12901, 25059, 25802, 50118, 743, 49375, 1486, 47889, 2972, 44917, 5944, 38973, 11888, 27085, 23776, 3309, 47552, 6618, 44243, 13236, 37625, 26472, 24389, 2083, 48778, 4166, 46695, 8332, 42529, 16664, 34197, 33328, 17533, 15795, 35066, 31590, 19271, 12319, 38542, 24638, 26223, 49276, 1585, 47691, 3170, 44521, 6340, 38181, 12680, 3, 50858, 5, 50856, 9, 50854, 7, 50852, 17, 50846, 15, 50844, 33, 50830, 31, 50828, 65, 5

### Trade-off Algorithm: 2025/02/22
* 此解出來的數字為位置及正負號 EX: $-1$ 代表 $-2^0$ , $-5$ 代表 $-2^4$ 以此類推

In [10]:
### 給一個R, 找錯誤位置h1及h2(包含錯誤正負號)
###
import math
import numpy as np

h1 = 0; h2 = 0; h_temp = 0
possible_h1 = []; possible_h2 = []; possible_AWE = []
output = []; possible_SE_h1=[]


for R, error in zip(remainder, new_e_total): 
    found = False                                       # 旗標，用來標示是否要跳出 for H 循環 
    for H in range(0,bits):                              
        for s in range(-1,2,2):                       
            h1 = s*(H+1)
            R1 = h_to_R[h1]
            R2 = R - R1
            if R2 < 0:
                R2 = A + R2                    
            h2 = R_to_h[R2]                  
            if (h2 != 0):
                s2 = get_sign(h2)                     
                possible_h1.append(h1)                  # 若找不到對應的h2,h2定為0(函數內容已定義)
                possible_h2.append(h2)                  # 代表錯誤可能是三個或以上(multiple errors)
                output.append((error - (s*2**(abs(h1)-1)) - (s2 * 2**(abs(h2)-1))) // A)
                found = True
                break
        if found:
            break

print(possible_h1)
print(possible_h2)
print("更正後的值:",output)
print("更正後的值是否和原data相同:", data == output.pop())
print(np.size(possible_h1))
print(np.size(possible_h2))
print("輸入的 error data的總數是否和經過演算法後相同:", error_total_count == np.size(possible_h1))
# print(possible_SE_h1)

[-1, 1, 1, -1, 2, -2, 3, -3, 4, -4, 5, -5, 6, -6, 7, -7, 8, -8, 9, -9, 10, -10, 11, -11, 12, -12, 13, -13, 14, -14, 15, -15, 16, -16, 17, -17, 18, -18, 19, -19, 20, -20, 21, -21, 22, -22, 23, -23, 24, -24, 25, -25, 26, -26, 27, -27, 28, -28, 29, -29, 30, -30, 31, -31, 32, -32, 33, -33, 34, -34, 35, -35, 36, -36, 37, -37, 38, -38, 39, -39, 40, -40, 41, -41, 42, -42, 43, -43, 44, -44, 45, -45, 46, -46, 47, -47, 48, -48, 49, -49, 50, -50, 51, -51, 52, -52, 53, -53, 54, -54, 55, -55, 56, -56, 57, -57, 58, -58, 59, -59, 60, -60, 61, -61, 62, -62, 63, -63, 64, -64, 65, -65, 66, -66, 67, -67, -1, -1, 1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 

#### 透過演算法發現"所有"的 Single AWE 都可以被拆成 Double AWE, 因此 deltaQ-table 不需要 

In [11]:
AWE_1_from_h = [2**(abs(n)-1) if n > 0 else -2**(abs(n)-1) for n in possible_h1]
AWE_2_from_h = [2**(abs(n)-1) if n > 0 else -2**(abs(n)-1) for n in possible_h2]
Total_AWE_from_h = [a+b for a, b in zip(AWE_1_from_h, AWE_2_from_h)]

print("h1的AWE為:",AWE_1_from_h)
print("h2的AWE為:",AWE_2_from_h)
print("解出來的AWE值為:",Total_AWE_from_h)
print("解出來的AWE是否和原AWE相同:", new_AWE == Total_AWE_from_h)

h1的AWE為: [-1, 1, 1, -1, 2, -2, 4, -4, 8, -8, 16, -16, 32, -32, 64, -64, 128, -128, 256, -256, 512, -512, 1024, -1024, 2048, -2048, 4096, -4096, 8192, -8192, 16384, -16384, 32768, -32768, 65536, -65536, 131072, -131072, 262144, -262144, 524288, -524288, 1048576, -1048576, 2097152, -2097152, 4194304, -4194304, 8388608, -8388608, 16777216, -16777216, 33554432, -33554432, 67108864, -67108864, 134217728, -134217728, 268435456, -268435456, 536870912, -536870912, 1073741824, -1073741824, 2147483648, -2147483648, 4294967296, -4294967296, 8589934592, -8589934592, 17179869184, -17179869184, 34359738368, -34359738368, 68719476736, -68719476736, 137438953472, -137438953472, 274877906944, -274877906944, 549755813888, -549755813888, 1099511627776, -1099511627776, 2199023255552, -2199023255552, 4398046511104, -4398046511104, 8796093022208, -8796093022208, 17592186044416, -17592186044416, 35184372088832, -35184372088832, 70368744177664, -70368744177664, 140737488355328, -140737488355328, 2814749767106

### DEC ${r}$-LUT:從餘數 ${r}$ 解回位置 ${l}$

In [12]:
#### DEC_AN Decoder
# 2025/4/8
#-------------------------------------
# Used to do DEC, but corrected errors by location, not AWE
# input: remainder, output: location
Bw = bits
Bn = bits_needed(data)
BA = bits_needed(A)
B_location = bits_needed(bits)
#-------------------------------------
fn = 'DEC_rLUT' + str(Bn) + "bits" +'.v'
f = open(fn, 'w')

f.write('// Product (AN) Code DEC r-LUT\n')
f.write('// ' + fn + '\n')
f.write('// Used to do DEC, but corrected errors by locations, not AWE\n')
f.write('// Received remainder r, output two error locations.\n')
f.write('module DEC_rLUT%dbits(r, l_1, l_2);\n'% (Bn))
f.write('input \t[%d:0]\tr;\n' % (BA-1))
f.write('output\treg\tsigned\t[%d:0]\tl_1;\n' % (B_location))
f.write('output\treg\tsigned\t[%d:0]\tl_2;\n' % (B_location))

f.write('always@(*) begin\n')
f.write('\tcase(r)\n')
for i in range(error_total_count):
    f.write("\t\t%d: begin l_1 = %+d;\n" % (remainder[i] ,possible_h1[i]))
    f.write("\t\t\t\t l_2 = %+d; end\n" % (possible_h2[i]))
f.write("\t\tdefault: begin l_1 = 0;\n")
f.write("\t\t\t\t\t   l_2 = 0; end\n")
f.write('\tendcase\n')
f.write('end\n\n')

f.write('endmodule\n')
f.close();
print('Module file %s is generated.\n' % (fn))

Module file DEC_rLUT52bits.v is generated.

